This Jupyter notebook calculates the ring parameter as explained in the report for a sequence of images or a single image

### Load required modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
from skimage.filters import median, threshold_multiotsu
from skimage.feature import canny
from skimage.io import imread
from skimage.measure import profile_line, regionprops, label
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.morphology import disk

### Define some convenience functions

Detect circles in the image, zoom into the most prominent one

In [ ]:
def detectCircles(img, radii=np.arange(20,250,10)):
    # get the shape of the image
    imgShape = img.shape
    
    # detect edges in the image using canny edge detection
    imgMed = median(img)
    edges = canny(imgMed, sigma=30)

    # detect circles in the image using a circular hough transform
    hough_res = hough_circle(edges, radii, normalize=False)

    # select the most prominent circle
    accums, cx, cy, radius = hough_circle_peaks(hough_res, radii, num_peaks=1, total_num_peaks=1)

    # zoom into the detected circle
    imgCenter = np.array([imgShape[0]/2, imgShape[1]/2])
    pad = 100
    imgRadius = radius + pad
    ymax = int(cy + imgRadius)
    ymin = int(cy - imgRadius)
    xmax = int(cx + imgRadius)
    xmin = int(cx - imgRadius)
    imgZoom = img[ymin:ymax, xmin:xmax].astype('uint8')
    
    return imgZoom

Check if a ring is visible enough in the image

In [ ]:
def detectRing(img, radii=np.arange(20,250,10)):
    # detect a circle in the image and zoom in
    imgZoom = detectCircles(img, radii)
    
    # apply a median filter to remove noise
    imgMed = median(imgZoom, disk(3))
    
    # threshold the image into three regions
    thresh = threshold_multiotsu(imgMed)
    imgThresh = np.digitize(imgMed, thresh)
    
    # label regions in the image and extract region properties
    backgroundLabel = imgThresh[0,0]
    labels = label(imgThresh, background=backgroundLabel, connectivity=1)
    rp = regionprops(labels, intensity_image=imgZoom)
    
    # calculate the ratio of each region area to the regions filled in area
    areaRatios = []
    for region in rp:
        area = region.area
        area_filled = region.area_filled
        areaRatios.append(area/area_filled)
    
    # reject the image if the ratio of areas is above 0.5, determined by analysing the given images
    areaMin = min(areaRatios)
    if areaMin > 0.5:
        return None
    else:
        return imgZoom

Process the image to ensure the ring is dark and background is bright

In [ ]:
def processImage(img, radii=np.arange(20,250,10)):
    # detect a circle in the image and zoom in
    imgZoom = detectRing(img, radii)
    if (imgZoom is not None):
        # if image not rejected, ensure the rings are dark
        
        # find max and min pixel values
        pixValMax = np.max(imgZoom)
        pixValMin = np.min(imgZoom)
        
        # determine which of these is the peak by comparing to the median pixel value
        medPixVal = np.median(imgZoom)
        maxDist = np.abs(pixValMax-medPixVal)
        minDist = np.abs(pixValMin-medPixVal)
        
        # the actual peak value is whichever value is furthest away from the median value
        # if the peak is bright, invert the image
        if (maxDist > minDist):
            imgZoom = ~imgZoom
        return imgZoom
    return None

Calculate pixel intensities at the minima, in the ring interior, and outside the ring

In [ ]:
def calculateIntensities(img, startPoint, endPoint, tol=10):
    # compute a line profile across the image
    lp = profile_line(img, startPoint, endPoint)
    lpLen = len(lp)
    
    # find the values and locations of the first and second pixel minima (i.e the ring itself)
    pixMinLeft = np.min(lp[0:int(lpLen/2)])
    pixMinLeftInd = np.where(lp[0:int(lpLen/2)] == pixMinLeft)[0][0]
    pixMinRight = np.min(lp[int(lpLen/2):lpLen])
    pixMinRightInd = np.where(lp[int(lpLen/2):lpLen] == pixMinRight)[0][0] + int(lpLen/2)
    # calculate the average of the pixel minima
    pixMin = int(np.mean([pixMinLeft, pixMinRight]))
    
    # compute median pixel values in the exterior and interior regions of the ring 
    # use a tolerance to ensure the median is computed far enough away from each peak
    medPixValInt = np.median(lp[pixMinLeftInd+tol:pixMinRightInd-tol])
    medPixValExt = np.median(np.concatenate([lp[0:pixMinLeftInd-tol],lp[pixMinRightInd+tol:lpLen]]))
    
    return medPixValInt, medPixValExt, pixMin

Get average of pixel intensities for multiple line profiles (20 by default)

In [ ]:
def averageIntensities(img, num=20):
    pixValsInt = 0
    pixValsExt = 0
    pixValsMin = 0
    imgShape = img.shape
    yCen = int(imgShape[0]/2)
    xCen = int(imgShape[1]/2)
    
    # compute the angles needed for the required number of line profiles
    angles = np.linspace(0, 180, num+1)
    angles = np.delete(angles,-1)
    angles = np.deg2rad(angles)
    
    # find the start and end point of each line
    startPoints = []
    endPoints = []
    for a in angles:
        xStart = xCen - xCen*np.cos(a)
        xEnd = xCen + xCen*np.cos(a)
        yStart = xCen - yCen*np.sin(a)
        yEnd = yCen + yCen*np.sin(a)
        startPoints.append([yStart,xStart])
        endPoints.append([yEnd,xEnd])
    
    # compute average pixel intensities across each line profile for the ring, exterior region, and interior region
    for i in range(num):
        intensities = calculateIntensities(img, startPoints[i], endPoints[i])
        pixValsInt += intensities[0]
        pixValsExt += intensities[1]
        pixValsMin += intensities[2]
        
    pixValsInt = pixValsInt/num
    pixValsExt = pixValsExt/num
    pixValsMin = pixValsMin/num
    return pixValsInt, pixValsExt, pixValsMin

Calculate the ring parameter for an image, given weighting factors $w_{ext}$, $w_{int}$ (0.75, 0.25 by default)

In [ ]:
# calculate ring parameter
def ringParameter(img, wExt=0.75, wInt=0.25, radii=np.arange(20,250,10)):
    # process image so ring is dark
    imgProcessed = processImage(img, radii)
    if (imgProcessed is not None):
        intensities = averageIntensities(imgProcessed)
        return (1 - intensities[2]*((wExt/intensities[1])+(wInt/intensities[0])))
    else:
        return 0

### Analyse the given images

To analyse a *single* image, rename the image to "testImage.jpg" and place the image into the **same directory as this Jupyter notebook**. Then run the following code block:

In [ ]:
# load the image
image = imread('testImage.jpg', as_gray=True)*255

# analyse the image
ringParam = ringParameter(image)
if ringParam != 0:
    print('Ring has ring parameter {}'.format(ringParam))
else:
    print('Ring not clearly visible')

To analyse *multiple* images, create a folder named "Images" in the **same directory as this Jupyter notebook**, and place the images into this folder. Then run the following code block:

In [ ]:
# set path to input image sequence
input_dir  = "Images/*"
imgArr = []

# load the images
for fname in glob.glob(input_dir):
    im = imread(fname, as_gray=True)*255
    imgArr.append(im)

# analyse the images
ringParams = []
start = time.time()
for img in imgArr:
    ringParams.append(ringParameter(img))
end = time.time()

print('Done, took {} minutes'.format(np.round((end-start)/60,3)))
print(ringParams)